In [1]:
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from datetime import datetime
import re

# Function to extract relevant events
def extract_events(input_text):
    lines = input_text.strip().split('\n')
    events = []
    project_created_time = None

    date_pattern = re.compile(r'\d{2}/\d{2}/\d{4}')
    time_pattern = re.compile(r'\d{1,2}:\d{2} (?:AM|PM)')

    current_date = None
    current_time = None

    for index, line in enumerate(lines):
        line = line.strip()

        # Check for date
        if date_pattern.match(line):
            current_date = line
            continue

        # Check for time
        if time_pattern.match(line):
            current_time = line
            continue

        # Check for project created
        if 'created project' in line.lower():
            project_created_time = f"{current_date} {current_time}"
            events.append((project_created_time, 'Project created'))
            continue

        # Check for status change events
        if 'changed the status to' in line.lower():
            event_time = f"{current_date} {current_time}"
            status = line.split('changed the status to')[-1].strip()
            events.append((event_time, status))
            continue

        # Check for assigned to pilot events
        if 'assigned to pilot' in line.lower():
            event_time = f"{current_date} {current_time}"
            pilot_name = None
            for i in range(index + 1, len(lines)):
                if lines[i].strip():  # Check if the line is not empty
                    pilot_name = lines[i].strip()
                    break
            if pilot_name:
                events.append((event_time, f'Assigned to Pilot ({pilot_name})'))
            continue

    return events

# Project ID textbox
project_id_box = widgets.Text(description='Project ID:', placeholder='Enter project ID', layout=widgets.Layout(width='100%'))

# Textbox for user input
text_box = widgets.Textarea(value='', placeholder='Paste your text here', description='Activities:', layout=widgets.Layout(width='100%', height='200px'))

# Button to generate the events
button_generate_events = widgets.Button(description="Generate Events", button_style='success')

# Editable textbox for generated events
events_box = widgets.Textarea(value='', placeholder='Review and edit events here', description='Events:', layout=widgets.Layout(width='100%', height='200px'))

# Button to generate the plot
button_generate_plot = widgets.Button(description="Generate Plot", button_style='success')

# Output widgets for the events table and plot
output_table = widgets.Output()
output_plot = widgets.Output()

def generate_events(event):
    with output_table:
        clear_output()
        # Extract relevant events from the text box
        input_text = text_box.value.strip()
        extracted_events = extract_events(input_text)
        
        # Convert extracted events to the data format
        data = "\n".join([f"{datetime.strptime(event[0], '%m/%d/%Y %I:%M %p').strftime('%m/%d/%y %H:%M')} {event[1]}" for event in extracted_events])
        
        # Display the extracted events in the editable textbox
        events_box.value = data

def generate_plot(event):
    with output_plot:
        clear_output()
        # Read the events from the events box
        events = events_box.value.strip().split('\n')
        event_list = []
        
        # Parse the events
        for event in events:
            parts = event.split(' ', 2)
            date_time_str = f"{parts[0]} {parts[1]}"
            date_time = datetime.strptime(date_time_str, '%m/%d/%y %H:%M')
            description = parts[2]
            event_list.append((date_time, description))

        # Sort events by date
        event_list.sort(key=lambda x: x[0])

        # Display the events list in a table
        event_table = """
        <table style='width:50%; border:1px solid black; border-collapse:collapse;'>
            <tr style='border:1px solid black;'>
                <th style='border:1px solid black; padding:4px; text-align:left;'>Time</th>
                <th style='border:1px solid black; padding:4px; text-align:right;'>Event</th>
            </tr>
        """
        for event in event_list:
            event_table += f"""
            <tr style='border:1px solid black;'>
                <td style='border:1px solid black; padding:4px; text-align:left;'>{event[0].strftime('%Y-%m-%d %H:%M')}</td>
                <td style='border:1px solid black; padding:4px; text-align:right;'>{event[1]}</td>
            </tr>
            """
        event_table += "</table>"
        
        display(HTML(event_table))

        # Convert dates to datetime objects and sort events by datetime
        dates = [date for date, _ in event_list]
        labels = [label for _, label in event_list]

        # Create the timeline plot
        fig, ax = plt.subplots(figsize=(14, 7))  # Adjust the figure size to fit the layout
        ax.plot(dates, range(len(dates)), marker='o', linestyle='-', color='green')  # Line goes up

        # Redraw to ensure correct axis limits and determine the midpoint of the x-axis
        plt.draw()
        midpoint = ax.get_xlim()
        midpoint = midpoint[0] + (midpoint[1] - midpoint[0]) / 2
        midpoint_date = mdates.num2date(midpoint).replace(tzinfo=None)

        # Add labels to the timeline based on their position relative to the midpoint
        for i, (date, label) in enumerate(zip(dates, labels)):
            alignment = 'left' if date < midpoint_date else 'right'
            ax.text(date, i, f"     {label}" if alignment == 'left' else f"{label}     ", 
                    ha=alignment, fontsize=12, va='center')

        ax.xaxis.set_major_locator(mdates.DayLocator(interval=7))
        ax.xaxis.set_major_formatter(mdates.DateFormatter("%b %d"))
        plt.setp(ax.get_xticklabels(), rotation=45, ha='right', fontsize=12)

        # Set grid, labels and title
        project_id = project_id_box.value
        ax.set_title(project_id, fontsize=16)

        ax.grid(True)
        ax.set_xlabel("Date", fontsize=14)
        ax.set_yticks(range(len(dates)))
        ax.set_yticklabels([])  # Remove y-axis labels

        plt.tight_layout()
        plt.show()

button_generate_events.on_click(generate_events)
button_generate_plot.on_click(generate_plot)

layout = widgets.VBox([
    project_id_box,
    text_box,
    button_generate_events,
    events_box,
    button_generate_plot,
    output_table,
    output_plot
])

display(layout)
